In [ ]:
#OAM Diffraction order analysis
!pip -q install opencv-python-headless numpy matplotlib scipy

import numpy as np, cv2, matplotlib.pyplot as plt
from google.colab import files
from collections import defaultdict
import os

#geometrical and parameters to tweak
USE_RED_IF_COLOR      = True
CENTER_CROP_TO_COMMON = True
AVERAGING_METHOD      = "mean"

APPLY_GAUSSIAN_BLUR   = True
GAUSS_KSIZE           = 3
APPLY_TOPHAT          = True
TOPHAT_KSIZE          = 51

THRESH_METHOD         = "percent_of_max"  # "otsu","adaptive","percent_of_max"
PERCENT_OF_MAX        = 0.12


MORPH_CLOSE           = True
MORPH_CLOSE_KSIZE     = 5


MIN_BLOB_AREA         = 60
MIN_REL_POWER         = 0.03
PERP_TOL_PX           = 14


MAX_ABS_ORDER         = 2
SYMMETRIZE_PM_PAIRS   = True
ENFORCE_DECR_ABS_M    = True

# Profiles & windows (for +1-only analysis)
ANG_BINS              = 360
PLUS1_BAND_FRAC       = 0.08   # annulus half-width as fraction of +1 radius (>=4 px)
PLUS1_MIN_BAND_PX     = 4
LINE_AVG_HALFWIDTH    = 2      # average ± this many pixels for line profiles
LINE_ROI_HALFWIDTH    = 40     # show cross-sections within ± this px around +1 centroid

SAVE_DIR              = "oam_plots"
SAVE_SUMMARY_NAME     = "summary_4plots.png"
os.makedirs(SAVE_DIR, exist_ok=True)

#processing functions
def imread_gray01(filebytes):
    arr = np.frombuffer(filebytes, np.uint8)
    img = cv2.imdecode(arr, cv2.IMREAD_UNCHANGED)
    if img is None:
        raise ValueError("Could not decode image.")
    if img.ndim == 2:
        g = img.astype(np.float32)
    else:
        if USE_RED_IF_COLOR:
            g = img[:, :, 2].astype(np.float32)
        else:
            g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.float32)
    g -= g.min()
    m = g.max()
    if m > 0: g /= m
    return g

def center_crop_to_common_size(imgs):
    if not imgs: return imgs
    Hc = min(im.shape[0] for im in imgs)
    Wc = min(im.shape[1] for im in imgs)
    out = []
    for im in imgs:
        H, W = im.shape
        y0 = (H - Hc)//2
        x0 = (W - Wc)//2
        out.append(im[y0:y0+Hc, x0:x0+Wc].copy())
    return out

def preprocess(g):
    out = g.copy()
    if APPLY_GAUSSIAN_BLUR and GAUSS_KSIZE % 2 == 1 and GAUSS_KSIZE >= 3:
        out = cv2.GaussianBlur(out, (GAUSS_KSIZE, GAUSS_KSIZE), 0)
    if APPLY_TOPHAT and TOPHAT_KSIZE % 2 == 1 and TOPHAT_KSIZE >= 3:
        k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (TOPHAT_KSIZE, TOPHAT_KSIZE))
        out = cv2.morphologyEx((out * 255).astype(np.uint8), cv2.MORPH_TOPHAT, k).astype(np.float32) / 255.0
    out -= out.min()
    m = out.max()
    if m > 0: out /= m
    return out

def combine(images, method="mean"):
    st = np.stack(images, 0)
    return np.median(st, 0) if method == "median" else np.mean(st, 0)

def find_center(img):
    y0, x0 = np.unravel_index(np.argmax(img), img.shape)
    win = 25
    y1 = max(0, y0 - win); y2 = min(img.shape[0], y0 + win + 1)
    x1 = max(0, x0 - win); x2 = min(img.shape[1], x0 + win + 1)
    patch = img[y1:y2, x1:x2]
    yy, xx = np.mgrid[y1:y2, x1:x2]
    tot = patch.sum()
    if tot <= 0: return float(y0), float(x0)
    cy = (yy * patch).sum() / tot; cx = (xx * patch).sum() / tot
    return float(cy), float(cx)

def threshold_image(img):
    g8 = (img * 255).astype(np.uint8)
    if THRESH_METHOD == "otsu":
        _, bw = cv2.threshold(g8, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    elif THRESH_METHOD == "adaptive":
        bw = cv2.adaptiveThreshold(g8, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY, 101, -5)
    else:
        thr = int(PERCENT_OF_MAX * 255)
        _, bw = cv2.threshold(g8, thr, 255, cv2.THRESH_BINARY)
    if MORPH_CLOSE and MORPH_CLOSE_KSIZE >= 3:
        k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (MORPH_CLOSE_KSIZE, MORPH_CLOSE_KSIZE))
        bw = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, k)
    k3 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    bw = cv2.morphologyEx(bw, cv2.MORPH_OPEN, k3)
    return bw

def label_blobs(bw, min_area=50):
    num, lab = cv2.connectedComponents(bw, 8)
    blobs = []
    for idx in range(1, num):
        m = (lab == idx)
        area = int(m.sum())
        if area < min_area: continue
        ys, xs = np.nonzero(m)
        blobs.append({
            "mask": m,
            "area": area,
            "centroid": (float(ys.mean()), float(xs.mean())),
            "sum_intensity": 0.0
        })
    return blobs

def assign_orders(blobs, center):
    if not blobs: return {}, np.array([1.0, 0.0])
    cy, cx = center
    offs = np.array([[b["centroid"][1] - cx, b["centroid"][0] - cy] for b in blobs], float)
    offs -= offs.mean(0, keepdims=True)
    _, _, Vt = np.linalg.svd(offs, full_matrices=False)
    axis = Vt[0]
    proj = offs @ axis
    dists = np.linalg.norm(offs, axis=1)
    zero_idx = int(np.argmin(dists))
    zpos = proj[zero_idx]
    idx = np.argsort(proj)
    right = [i for i in idx if proj[i] > zpos]
    left  = [i for i in idx if proj[i] < zpos]
    orders = {zero_idx: 0}
    for k, i in enumerate(sorted(right, key=lambda i: abs(proj[i] - zpos)), 1): orders[i] = +k
    for k, i in enumerate(sorted(left,  key=lambda i: abs(proj[i] - zpos)), 1): orders[i] = -k
    return {i: orders.get(i) for i in range(len(blobs))}, axis

def radial_profile_masked(img, center, mask):
    H, W = img.shape; cy, cx = center
    y, x = np.indices((H, W))
    r = np.sqrt((x - cx)**2 + (y - cy)**2).astype(np.int32)
    rr = r[mask]; vv = img[mask]
    tbin = np.bincount(rr.ravel(), vv.ravel())
    cnt  = np.bincount(rr.ravel())
    return tbin / np.maximum(cnt, 1)

def angular_profile(img, center, r_min, r_max, nbins=360):
    H, W = img.shape; cy, cx = center
    y, x = np.indices((H, W))
    r  = np.sqrt((x - cx)**2 + (y - cy)**2)
    th = np.arctan2(y - cy, x - cx)
    mask = (r >= r_min) & (r <= r_max)
    edges = np.linspace(-np.pi, np.pi, nbins + 1)
    sums, _  = np.histogram(th[mask], bins=edges, weights=img[mask])
    cnts, _  = np.histogram(th[mask], bins=edges)
    prof = sums / np.maximum(cnts, 1)
    centers_deg = 0.5 * (edges[:-1] + edges[1:]) * 180/np.pi
    return centers_deg, prof

# Algorithm
print("Select one or more images (JPG/PNG) of the SAME pattern to combine:")
uploads = files.upload()
if not uploads: raise SystemExit("No files uploaded.")

imgs = []
for name, fb in uploads.items():
    try:
        g = imread_gray01(fb)
        imgs.append(g)
        print(f"Loaded: {name}  shape={g.shape}")
    except Exception as e:
        print(f"Skipping {name}: {e}")
if not imgs: raise SystemExit("No valid images.")

# unify sizes
if CENTER_CROP_TO_COMMON:
    imgs = center_crop_to_common_size(imgs)

# preprocess + combine
imgs = [preprocess(im) for im in imgs]
combo = combine(imgs, method=AVERAGING_METHOD)
print(f"Combined {len(imgs)} image(s) by {AVERAGING_METHOD} averaging.")

# center
center = find_center(combo)
print(f"Detected center (y, x) = ({center[0]:.1f}, {center[1]:.1f})")

# blobs
bw    = threshold_image(combo)
blobs = label_blobs(bw, min_area=MIN_BLOB_AREA)
for b in blobs: b["sum_intensity"] = combo[b["mask"]].sum()

# power floor
max_pow = max([b["sum_intensity"] for b in blobs], default=1.0)
blobs   = [b for b in blobs if b["sum_intensity"] >= MIN_REL_POWER * max_pow]

# on-axis gate
order_map, axis_vec = assign_orders(blobs, center)
perp_vec = np.array([-axis_vec[1], axis_vec[0]])
def perp_dist(b):
    cy, cx = center
    off = np.array([b["centroid"][1] - cx, b["centroid"][0] - cy])
    return abs(off @ perp_vec)
blobs = [b for b in blobs if perp_dist(b) <= PERP_TOL_PX]

#order assignment
order_map, axis_vec = assign_orders(blobs, center)

#Modes regularization
order_power = defaultdict(float)
for i, b in enumerate(blobs):
    m = order_map.get(i, None)
    if m is None: continue
    if MAX_ABS_ORDER is not None and abs(m) > MAX_ABS_ORDER: continue
    order_power[m] += float(b["sum_intensity"])


for m in range(-2, 3):
    order_power.setdefault(m, 0.0)

def _warn(msg):
    print("[Modes check]", msg)

# Diagnostic
if abs(order_power[+2] - order_power[-2]) > 0.05 * max(1.0, order_power[0]):
    _warn(f"±2 asymmetry detected: +2={order_power[+2]:.3g}, -2={order_power[-2]:.3g}")
if abs(order_power[+1] - order_power[-1]) > 0.05 * max(1.0, order_power[0]):
    _warn(f"±1 asymmetry detected: +1={order_power[+1]:.3g}, -1={order_power[-1]:.3g}")
if (order_power[+2] > order_power[+1]) or (order_power[-2] > order_power[-1]):
    _warn("|m|=2 appears stronger than |m|=1 (before adjustment).")


if SYMMETRIZE_PM_PAIRS:
    p1 = 0.5 * (order_power[+1] + order_power[-1])
    p2 = 0.5 * (order_power[+2] + order_power[-2])
    order_power[+1] = order_power[-1] = p1
    order_power[+2] = order_power[-2] = p2


if ENFORCE_DECR_ABS_M:
    p1 = max(order_power[+1], order_power[-1])
    order_power[+2] = min(order_power[+2], p1)
    order_power[-2] = min(order_power[-2], p1)


if abs(order_power[+2] - order_power[-2]) > 1e-12:
    _warn("±2 still asymmetric after symmetrization (numerical)")
if (order_power[+2] > order_power[+1]) or (order_power[-2] > order_power[-1]):
    _warn("|m|=2 still stronger than |m|=1 after regularization.")

orders_range  = list(range(-2, 3))
powers_sorted = [order_power[o] for o in orders_range]

#locate +1 spot
plus1_radii = []
plus1_centroid = None
for i, b in enumerate(blobs):
    m = order_map.get(i, None)
    if m == +1:
        by, bx = b["centroid"]; plus1_centroid = (by, bx)
        plus1_radii.append(np.hypot(bx - center[1], by - center[0]))
if not plus1_radii:
    for i, b in enumerate(blobs):
        m = order_map.get(i, None)
        if abs(m) == 1:
            by, bx = b["centroid"]
            plus1_radii.append(np.hypot(bx - center[1], by - center[0]))
if not plus1_radii:
    raise SystemExit("Could not locate the +1 ring. Try lowering threshold or filters.")

R1   = float(np.median(plus1_radii))
band = max(PLUS1_MIN_BAND_PX, PLUS1_BAND_FRAC * R1)
r1_min = max(0.0, R1 - band)
r1_max = R1 + band

H, W = combo.shape
yy, xx = np.indices((H, W))
rr = np.sqrt((xx - center[1])**2 + (yy - center[0])**2)
mask_plus1 = (rr >= r1_min) & (rr <= r1_max)

#figure
fig = plt.figure(figsize=(14, 10))
outer = fig.add_gridspec(2, 2, wspace=0.3, hspace=0.3)
ax_mode    = fig.add_subplot(outer[0, 0])
ax_radial  = fig.add_subplot(outer[0, 1])
ax_angular = fig.add_subplot(outer[1, 0])
inner      = outer[1, 1].subgridspec(1, 2, wspace=0.3)
ax_h       = fig.add_subplot(inner[0, 0])
ax_v       = fig.add_subplot(inner[0, 1])

# 1) Modes intensity (-2..+2)
ax_mode.bar(orders_range, powers_sorted)
ax_mode.set_xlabel("Diffraction order (m)")
ax_mode.set_ylabel("Integrated power (a.u.)")
ax_mode.set_title("Modes Intensity (−2 … +2)")

# 2) Radial intensity (ONLY +1 annulus)
rad_plus1 = radial_profile_masked(combo, center, mask_plus1)
ax_radial.plot(np.arange(len(rad_plus1)), rad_plus1)
ax_radial.set_xlabel("Radius (pixels) from center")
ax_radial.set_ylabel("Average intensity (a.u.)")
ax_radial.set_title("Radial Intensity (on +1 annulus)")

# 3) Angular intensity (ONLY +1 annulus)
ang_deg, ang_val = angular_profile(combo, center, r1_min, r1_max, nbins=ANG_BINS)
ax_angular.plot(ang_val, ang_deg)
ax_angular.set_ylabel("Azimuthal angle (degrees)")
ax_angular.set_xlabel("Average intensity (a.u.)")
ax_angular.set_title("Angular Intensity (on +1 annulus)")

# 4) Line cross-sections THROUGH +1 (local window)
if plus1_centroid is None:
    plus1_centroid = (center[0], center[1])
py, px = int(round(plus1_centroid[0])), int(round(plus1_centroid[1]))
y1 = max(0, py - LINE_AVG_HALFWIDTH); y2 = min(H, py + LINE_AVG_HALFWIDTH + 1)
x1 = max(0, px - LINE_AVG_HALFWIDTH); x2 = min(W, px + LINE_AVG_HALFWIDTH + 1)

horiz = combo[y1:y2, :].mean(axis=0)
xh0 = max(0, px - LINE_ROI_HALFWIDTH); xh1 = min(W, px + LINE_ROI_HALFWIDTH + 1)
ax_h.plot(np.arange(xh0, xh1) - px, horiz[xh0:xh1])
ax_h.set_title("Horizontal cross-section +1")
ax_h.set_xlabel("x (pixels relative to +1 centroid)")
ax_h.set_ylabel("Intensity (a.u.)")

vert = combo[:, x1:x2].mean(axis=1)
yv0 = max(0, py - LINE_ROI_HALFWIDTH); yv1 = min(H, py + LINE_ROI_HALFWIDTH + 1)
ax_v.plot(vert[yv0:yv1], np.arange(yv0, yv1) - py)
ax_v.set_title("Vertical cross-section +1")
ax_v.set_xlabel("Intensity (a.u.)")
ax_v.set_ylabel("y (pixels relative to +1 centroid)")

fig.tight_layout()
fig.savefig(os.path.join(SAVE_DIR, SAVE_SUMMARY_NAME), dpi=220)
plt.show()
print("Saved:", os.path.join(SAVE_DIR, SAVE_SUMMARY_NAME))


In [ ]:
#OAM Annular Grating analysis
#For future work, when enough images have been collected for annular grating fringe pattern from OAM = -2 to +2, we can train the small dataset using resnet
#A sample CNN based code has been given below
!pip -q install opencv-python-headless numpy matplotlib scipy
import numpy as np, cv2, matplotlib.pyplot as plt, os, csv
from google.colab import files
from scipy.signal import find_peaks, savgol_filter
from numpy.fft import rfft

#parameters
# Annulus center (ring) selection
R_SELECT_MODE          = "largest_prominence"
R_K_INDEX              = 1
R_SCALE                = 1.65


ANNULUS_HALF_FRAC      = 0.22                   # half-width as fraction of R
ANNULUS_MIN_HALFW_PX   = 10

# Robustness: suppress angular outliers (side streaks)
OUTLIER_SIGMA_CLIP     = 2.5

# Preprocessing
USE_RED_IF_COLOR       = True
CENTER_CROP_TO_COMMON  = True
AVERAGING_METHOD       = "mean"                 # "mean" or "median"
APPLY_GAUSS            = True; GAUSS_KSIZE = 3
APPLY_TOPHAT           = True; TOPHAT_KSIZE = 51

# Radial peak find (for R_raw)
RAD_PEAK_PROM_REL      = 0.05
RAD_PEAK_MIN_DIST_PX   = 6

# Angular profile binning & smoothing
ANG_BINS               = 720
BASELINE_REMOVE        = True
BASELINE_MOVING_WIN    = 61
SMOOTH_METHOD          = "savgol"
SAVGOL_WIN             = 15
SAVGOL_POLY            = 2
MOVING_WIN             = 9

# Harmonic (FFT) inference
KMAX_HARMONIC          = 6
HARMONIC_RATIO_MIN     = 1.25


PEAK_PROM_REL          = 0.15
PEAK_MIN_DIST_DEG      = 30

OUT_DIR                = "oam_donut_out"
os.makedirs(OUT_DIR, exist_ok=True)

def imread_gray01(filebytes):
    arr = np.frombuffer(filebytes, np.uint8)
    img = cv2.imdecode(arr, cv2.IMREAD_UNCHANGED)
    if img is None: raise ValueError("Could not decode image.")
    if img.ndim == 2:
        g = img.astype(np.float32)
    else:
        g = img[:,:,2].astype(np.float32) if USE_RED_IF_COLOR \
            else cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.float32)
    g -= g.min(); M = g.max()
    if M>0: g/=M
    return g

def center_crop_to_common(imgs):
    Hc = min(im.shape[0] for im in imgs)
    Wc = min(im.shape[1] for im in imgs)
    out=[]
    for im in imgs:
        H,W = im.shape
        y0=(H-Hc)//2; x0=(W-Wc)//2
        out.append(im[y0:y0+Hc, x0:x0+Wc].copy())
    return out

def preprocess(g):
    out = g.copy()
    if APPLY_GAUSS and GAUSS_KSIZE%2==1 and GAUSS_KSIZE>=3:
        out = cv2.GaussianBlur(out, (GAUSS_KSIZE,GAUSS_KSIZE), 0)
    if APPLY_TOPHAT and TOPHAT_KSIZE%2==1 and TOPHAT_KSIZE>=3:
        k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (TOPHAT_KSIZE,TOPHAT_KSIZE))
        out = cv2.morphologyEx((out*255).astype(np.uint8), cv2.MORPH_TOPHAT, k).astype(np.float32)/255.0
    out -= out.min(); M = out.max()
    if M>0: out/=M
    return out

def combine(imgs, method="mean"):
    st = np.stack(imgs,0)
    return np.median(st,0) if method=="median" else np.mean(st,0)

def find_center(img):
    y0,x0 = np.unravel_index(np.argmax(img), img.shape)
    win=25
    y1=max(0,y0-win); y2=min(img.shape[0],y0+win+1)
    x1=max(0,x0-win); x2=min(img.shape[1],x0+win+1)
    patch = img[y1:y2, x1:x2]; yy,xx=np.mgrid[y1:y2, x1:x2]
    s = patch.sum()
    if s<=0: return float(y0), float(x0)
    cy=(yy*patch).sum()/s; cx=(xx*patch).sum()/s
    return float(cy), float(cx)

def radial_profile(img, center):
    H,W = img.shape; cy,cx=center
    y,x = np.indices((H,W))
    r = np.sqrt((x-cx)**2 + (y-cy)**2)
    r_i = r.astype(np.int32)
    tbin = np.bincount(r_i.ravel(), img.ravel())
    cnt  = np.bincount(r_i.ravel())
    return tbin/np.maximum(cnt,1), r

def moving_average(y, k):
    if k<=1 or k%2==0: return y
    pad=k//2
    ypad=np.pad(y,(pad,pad),mode='edge')
    kern=np.ones(k)/k
    return np.convolve(ypad,kern,mode='valid')

def mad_z(x):
    med = np.median(x)
    mad = np.median(np.abs(x - med)) + 1e-9
    return 0.6745 * (x - med) / mad

print("Upload one or more images (same donut):")
uploads = files.upload()
if not uploads: raise SystemExit("No files uploaded.")
imgs=[]
for n,fb in uploads.items():
    try:
        g = imread_gray01(fb); imgs.append(g)
        print("Loaded:", n, g.shape)
    except Exception as e:
        print("Skipping", n, e)
if not imgs: raise SystemExit("No valid images.")
if CENTER_CROP_TO_COMMON: imgs = center_crop_to_common(imgs)
imgs = [preprocess(im) for im in imgs]
combo = combine(imgs, method=AVERAGING_METHOD)
H,W = combo.shape

center = find_center(combo)
rad_prof, rmap = radial_profile(combo, center)
r_pix = np.arange(len(rad_prof))

r_ignore = max(3, int(0.01*min(H,W)))
r_work = r_pix[r_ignore:]; p_work = rad_prof[r_ignore:]
p_s = savgol_filter(p_work, 11, 2) if len(p_work)>=11 else p_work
prom = RAD_PEAK_PROM_REL * (np.max(p_s) if p_s.size else 1.0)
pk_all, props = find_peaks(p_s, prominence=prom, distance=RAD_PEAK_MIN_DIST_PX)
if pk_all.size == 0:
    raise SystemExit("No radial peaks found. Try larger TOPHAT_KSIZE or smaller RAD_PEAK_PROM_REL.")

if R_SELECT_MODE == "first":
    R_raw = float(r_work[pk_all[0]])
elif R_SELECT_MODE == "largest_prominence":
    idx = int(np.argmax(props["prominences"]))
    R_raw = float(r_work[pk_all[idx]])
elif R_SELECT_MODE == "kth":
    k = int(np.clip(R_K_INDEX, 0, len(pk_all)-1))
    R_raw = float(r_work[pk_all[k]])
else:
    R_raw = float(r_work[pk_all[0]])

cy,cx = center
max_r_possible = np.sqrt(max(cy, H-cy-1)**2 + max(cx, W-cx-1)**2) - 2.0
R = min(R_SCALE * R_raw, max_r_possible)

halfw = max(ANNULUS_MIN_HALFW_PX, ANNULUS_HALF_FRAC * R)
rmin, rmax = max(0.0, R - halfw), min(max_r_possible, R + halfw)

# graph
yy,xx = np.indices(combo.shape)
rr = np.sqrt((xx-cx)**2 + (yy-cy)**2)
theta = np.arctan2(yy-cy, xx-cx)
mask = (rr>=rmin) & (rr<=rmax)

edges = np.linspace(-np.pi, np.pi, ANG_BINS+1)
vals, _ = np.histogram(theta[mask], bins=edges, weights=combo[mask])
cnts, _ = np.histogram(theta[mask], bins=edges)
ang = vals/np.maximum(cnts,1)
ang_deg = 0.5*(edges[:-1]+edges[1:]) * 180/np.pi

# outlier clipping
z = mad_z(ang)
ang_clipped = ang.copy()
ang_clipped[np.abs(z) > OUTLIER_SIGMA_CLIP] = np.median(ang)

if BASELINE_REMOVE and len(ang_clipped)>=BASELINE_MOVING_WIN:
    base = moving_average(ang_clipped, BASELINE_MOVING_WIN)
    ang_d = ang_clipped - base
else:
    ang_d = ang_clipped - np.mean(ang_clipped)

if SMOOTH_METHOD=="savgol" and len(ang_d)>=SAVGOL_WIN and SAVGOL_WIN%2==1:
    ang_s = savgol_filter(ang_d, SAVGOL_WIN, SAVGOL_POLY)
elif SMOOTH_METHOD=="moving" and len(ang_d)>=MOVING_WIN and MOVING_WIN%2==1:
    ang_s = moving_average(ang_d, MOVING_WIN)
else:
    ang_s = ang_d

A = np.abs(rfft(ang_s))
A = A / (A.max() + 1e-12)
harm_k = np.arange(len(A))
valid = (harm_k >= 1) & (harm_k <= KMAX_HARMONIC)
k_vals = harm_k[valid]
amps  = A[valid]

order = int(k_vals[np.argmax(amps)])
sorted_idx = np.argsort(amps)[::-1]
dom, second = amps[sorted_idx[0]], amps[sorted_idx[1]] if len(sorted_idx)>1 else 0.0
confident = (dom >= HARMONIC_RATIO_MIN * max(second, 1e-9))

#visualization
from scipy.signal import find_peaks
promA = 0.15 * (np.max(ang_s) - np.min(ang_s) + 1e-9)
min_dist_bins = max(1, int(PEAK_MIN_DIST_DEG * ANG_BINS / 360.0))
pk_idx, _ = find_peaks(ang_s, prominence=promA, distance=min_dist_bins)

#figure
overlay = cv2.cvtColor((combo*255).astype(np.uint8), cv2.COLOR_GRAY2BGR)
cv2.circle(overlay, (int(round(cx)), int(round(cy))), int(round(R_raw)), (0,255,0), 1)
cv2.circle(overlay, (int(round(cx)), int(round(cy))), int(round(R)), (0,200,0), 2)
cv2.circle(overlay, (int(round(cx)), int(round(cy))), int(round(rmin)), (255,0,0), 1)
cv2.circle(overlay, (int(round(cx)), int(round(cy))), int(round(rmax)), (255,0,0), 1)

fig = plt.figure(figsize=(13,9))
gs  = fig.add_gridspec(2,2, wspace=0.28, hspace=0.28)

ax0 = fig.add_subplot(gs[0,0])
ax0.imshow(overlay[...,::-1])
ax0.set_title(f"Annulus: R_raw={R_raw:.1f}px  →  R_scaled={R:.1f}px  |  halfw={halfw:.1f}px")
ax0.axis("off")

ax1 = fig.add_subplot(gs[0,1])
ax1.plot(ang,      ang_deg, lw=0.9, alpha=0.35, label="raw")
ax1.plot(ang_s,    ang_deg, lw=1.4, label="smoothed/detrended & clipped")
ax1.plot(ang_s[pk_idx], ang_deg[pk_idx], 'ro', ms=4, label="peaks (viz)")
ax1.set_xlabel("Average intensity (a.u.)")
ax1.set_ylabel("Azimuthal angle (deg)")
ax1.set_title("Angular intensity on annulus")
ax1.legend(loc="lower right")

ax2 = fig.add_subplot(gs[1,0])
ax2.plot(r_pix, rad_prof, label="radial mean")
ax2.axvline(R_raw, color='g', ls='--', lw=1, label="R_raw (peak)")
ax2.axvline(R,     color='g', ls='-',  lw=1.2, label="R_scaled")
ax2.fill_betweenx([0, rad_prof.max()], rmin, rmax, color='tab:blue', alpha=0.12, label="annulus")
ax2.set_xlabel("Radius from donut center (pixels)")
ax2.set_ylabel("Mean intensity (a.u.)")
ax2.set_title("Radial profile & chosen annulus")
ax2.legend()

ax3 = fig.add_subplot(gs[1,1])
ax3.bar(k_vals, amps, width=0.8)
ax3.set_xticks(k_vals)
ax3.set_xlabel("Harmonic index k")
ax3.set_ylabel("Normalized amplitude")
ax3.set_title(f"Harmonic spectrum of angular profile → inferred |ℓ| = {order}"
              + ("" if confident else " (low confidence)"))

fig.tight_layout()
png_path = os.path.join(OUT_DIR, "donut_harmonics_summary.png")
fig.savefig(png_path, dpi=220)
plt.show()

csv_path = os.path.join(OUT_DIR, "harmonics_table.csv")
with open(csv_path, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["k", "norm_amplitude"])
    for k,a in zip(k_vals, amps):
        w.writerow([int(k), float(a)])
print(f"Inferred |ell| = {order}  (dominant={dom:.3f}, next={second:.3f}, confident={confident})")
print("Saved figure:", png_path)
print("Saved spectrum CSV:", csv_path)

In [ ]:
#CNN Approach to Annular Grating dark fringe analysis using pre trained
#with the help of CNN there is no need to manually tweak the model as we did in the previous codes and that makes it more generalized and accurate in classification
#Once the dataset is large enough by collecting enough sample images of dark fringes, we can train the model from scratch for customization
# a reference paper talks about using CNN classification approach in free space for OAM based communication - Ye, J., Kang, H., Wang, H., Shen, C., Jahannia, B., Heidari, E., Asadizanjani, N., Miri, M.A., Sorger, V.J. and Dalir, H., 2023, September. Demultiplexing oam beams via fourier optical convolutional neural network. In Laser Beam Shaping XXIII (Vol. 12667, pp. 16-33). SPIE.
!pip -q install torch torchvision pandas pillow
import pandas as pd
from PIL import Image
from torchvision import transforms, models
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch, torch.nn as nn
from google.colab import files
import io
from sklearn.model_selection import train_test_split

# Custom dataset for OAM images
class OAMDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)  # CSV with columns [image_path, label]
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 0]
        label = int(self.data.iloc[idx, 1])
        # Convert label to 0..4 for 5 classes
        label = label + 2
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# Define image transformations (resize, normalize, etc.)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

#dataset upload .csv file
print("Please upload your CSV file (with columns X=images, Y=labels):")
uploaded = files.upload()
csv_path = list(uploaded.keys())[0]
print(f"Loaded dataset CSV: {csv_path}")

#training testing split (validation is 20 percent), can be used in the epoch loop to see training vs validation accuracy
df = pd.read_csv(csv_path)
dataset = OAMDataset(csv_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['Y'], random_state=42)
train_dataset = OAMDataset(train_df, transform=transform)
val_dataset   = OAMDataset(val_df, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=16, shuffle=False)
print(f"Train size: {len(train_dataset)}, Validation size: {len(val_dataset)}")

# Load a pre-trained ResNet18 model
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 5)  # 5 output classes (-2,-1,0,1,2)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training loop (outline)
num_epochs = 5
model.train()
for epoch in range(num_epochs):
    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# Save the trained model
torch.save(model.state_dict(), 'oam_cnn_model.pth')

# Inference on a new image
from torchvision import transforms

# Load the trained model weights
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(num_features, 5)
model.load_state_dict(torch.load('oam_cnn_model.pth'))
model = model.to(device)
model.eval()

# Prepare the image
img_path = 'new_oam_image.jpg'
image = Image.open(img_path).convert('RGB')
img_tensor = transform(image).unsqueeze(0).to(device)

# Predict OAM class
with torch.no_grad():
    output = model(img_tensor)
    _, pred = torch.max(output, 1)
oam_pred = pred.item() - 2
print("Predicted OAM value:", oam_pred)
